In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
spark=SparkSession.builder.appName('data_processing').getOrCreate()

Par rapport aux Udf standard de Python, les udf panda sont :
    - plus rapides
    - plus léger en terme de temps
    - plus léger en terme d'execution
    La différence entre les deux réside dans le fait qu'un UDF panda s'execute en bloc par bloc
    alors qu'un udf python s'execute ligne par ligne et par conséquent n'offre pas les avantages d'un framework distribué.

On en distingue trois types qui sont:
    ***scalar grouped map et grouped agg***


In [17]:
employees = spark.read.csv("employee.csv", header=True, inferSchema=True) #import du fichier employee.csv
# employees.select('salary').summary().show() à decommenter pour voir le min et le max
min_sal =23056
max_sal = 498069

def scaled_salary(salary):
    """
        Cette fonction permet de calculer l'echelonnage du salaire. 
        :param salary : le salaire dont nous devons calculer l'echelonnage
        :return le salaire échelonné
    """
    scaled_sal = (salary-min_sal)/(max_sal-min_sal)
    return scaled_sal

In [18]:
scaling_udf = pandas_udf(scaled_salary,DoubleType()) #definition de la structure de la fonction panda_udf
employees.withColumn("scaled_salary", scaling_udf(employees['salary'])).show() #ajout de la colonne scaled_salary qui contient l'echelonnage

+----------------+------+--------------------+--------------------+
|            name|salary|             company|       scaled_salary|
+----------------+------+--------------------+--------------------+
|Colette Guerrero|372384|            Dui Inc.|  0.7354072414860224|
|    Preston Wall|387228|Turpis Non Enim I...|  0.7666569125476566|
|   Cadman Bright|200634|Pellentesque A Corp.| 0.37383818969165056|
|  Cameran Cannon| 25171|Porttitor Sceleri...|0.004452509720786589|
| Camilla Edwards|395840|             Elit PC|  0.7847869426731479|
|     Keiko Garza|292461|         Ornare Inc.|  0.5671528989732912|
| Clinton Mcguire|157623|   Mauris Eu Limited| 0.28329119413573944|
|    Maile Phelps|168556|            Urna LLC|  0.3063074063236164|
|     Todd Oliver| 97660|       Augue Sed LLC| 0.15705675423619986|
|Stephen Alvarado|253303|Ac Arcu Incorporated|  0.4847172603697162|
|    Judah Farley| 85607|Mollis Nec Cursus...| 0.13168271184157065|
| Daphne Roberson|101749|         Euismod Ltd| 0